In [ ]:
import numpy as np
import gym
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
import random


In [ ]:
# Initialize the CartPole environment
env = gym.make('CartPole-v1')

# Display environment details
print("Observation Space:", env.observation_space)
print("Action Space:", env.action_space)


In [ ]:
class DQLAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        # Neural network to approximate Q-values
        model = Sequential([
            Dense(24, input_dim=self.state_size, activation='relu'),
            Dense(24, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss='mse')
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)  # Random action
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])  # Action with max Q-value

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [ ]:
# Initialize the agent
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQLAgent(state_size, action_size)

# Training parameters
episodes = 500
batch_size = 32

# Train the agent
for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0

    for time in range(500):
        action = agent.act(state)  # Select action
        next_state, reward, done, _ = env.step(action)  # Take action
        total_reward += reward
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)  # Store experience
        state = next_state
        if done:
            print(f"Episode: {e + 1}/{episodes}, Score: {time}, Epsilon: {agent.epsilon:.2f}")
            break
        agent.replay(batch_size)  # Train the agent with replay memory


In [ ]:
# Test the trained agent
agent.epsilon = 0.0  # Disable exploration for testing
for e in range(5):  # Test for 5 episodes
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0

    for time in range(500):
        env.render()  # Render the environment
        action = agent.act(state)  # Predict action
        next_state, reward, done, _ = env.step(action)  # Take action
        total_reward += reward
        next_state = np.reshape(next_state, [1, state_size])
        state = next_state
        if done:
            print(f"Test Episode: {e + 1}, Total Reward: {total_reward}")
            break
env.close()


In [ ]:
# Save the trained model
agent.model.save('dql_cartpole_model.h5')
print("\nModel saved successfully!")

# Load the trained model
from tensorflow.keras.models import load_model
loaded_model = load_model('dql_cartpole_model.h5')
print("\nModel loaded successfully!")
